### Car Price Prediction using Random Forest Regressor

In [22]:
### import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [39]:
## dataset

df = pd.read_csv('cardekho.csv')

df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [40]:
### info of dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [41]:
### check for null values
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [42]:
## remove columns

df.drop(columns=['Unnamed: 0','car_name','brand'],axis=1,inplace=True)

In [43]:
## unique values in model column

df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [44]:
## categorical features

cat_features = [col for col in df.columns if df[col].dtype == 'object']

## numerical features

num_features = [col for col in df.columns if df[col].dtype != 'object']

## discrete features

discrete_features = [col for col in num_features if len(df[col].unique())<25]

## continuous features

contnuous_features = [col for col in num_features if col not in discrete_features]

print('total features: \n', len(df.columns))
print('categorical features: \n', len(cat_features))
print('numerical features: \n', len(num_features))
print('discrete features: \n', len(discrete_features))
print('continuous features: \n', len(contnuous_features))

total features: 
 11
categorical features: 
 4
numerical features: 
 7
discrete features: 
 2
continuous features: 
 5


In [45]:
## independent and dependent features
X = df.drop(columns=['selling_price'], axis=1)
y = df['selling_price']

### Feature Encoding and scaling

In [46]:
## label encoding for model column

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X['model'] = le.fit_transform(X['model'])
X

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
15406,117,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5
15407,42,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7
15408,77,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5
15409,114,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7


In [47]:
### column transformer

num_features = X.select_dtypes(exclude='object').columns
one_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

encoder = OneHotEncoder(drop='first',sparse_output=False)
scaler = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("onehotencoder",encoder, one_columns),
        ("Standard Scaler", scaler, num_features)
    ], remainder='passthrough'
)

X = preprocessor.fit_transform(X)

In [48]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [49]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [50]:
### Model training

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [54]:
## create a function to evaluate model

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return mae, mse, r2


## model training

models = {
    'Linear Regression': LinearRegression(),
    "Lasso": Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    "DEcision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    ## make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    ## evaluate models
    model_train_mae, model_train_mse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for training set')
    print(f"MAE: {model_train_mae:.4f}")
    print(f"MSE: {model_train_mse:.4f}")
    print(f"r2: {model_train_r2:.4f}")

    print('Model performance for testing set')
    print(f"MAE: {model_test_mae:.4f}")
    print(f"MSE: {model_test_mse:.4f}")
    print(f"r2: {model_test_r2:.4f}")
    
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for training set
MAE: 268101.6071
MSE: 306756099359.7596
r2: 0.6218
Model performance for testing set
MAE: 279618.5794
MSE: 252550062888.5655
r2: 0.6645


Lasso
Model performance for training set
MAE: 268099.2226
MSE: 306756104248.3742
r2: 0.6218
Model performance for testing set
MAE: 279614.7461
MSE: 252549134806.7813
r2: 0.6645


Ridge
Model performance for training set
MAE: 268059.8015
MSE: 306756818740.9266
r2: 0.6218
Model performance for testing set
MAE: 279557.2169
MSE: 252540243247.9685
r2: 0.6645


K-Neighbors Regressor
Model performance for training set
MAE: 91425.6327
MSE: 106193229526.2816
r2: 0.8691
Model performance for testing set
MAE: 112526.3461
MSE: 64021344520.1508
r2: 0.9150


DEcision Tree
Model performance for training set
MAE: 5164.8199
MSE: 432524990.5364
r2: 0.9995
Model performance for testing set
MAE: 125224.4486
MSE: 93152535013.8979
r2: 0.8763


Random Forest Regressor
Model performance for training set
MAE: 39828.8918
MS

In [55]:
## hyperparameter tuning for KNN

knn_params = {
    'n_neighbors':[2,3,10,20,40,50]
}

## hyperparameter tuning for RF

rf_params = {
    'max_depth':[5,8,15,None,10],
    'max_features':[5,7,'auto',10],
    'min_samples_split':[2,8,15,20],
    'n_estimators':[100,200,500,1000]
}

In [56]:
### model list for hyper parameter tuning

randomcv_models = [('KNN', KNeighborsRegressor(),knn_params),
                   ('RF',RandomForestRegressor(),rf_params)
                   ]

from sklearn.model_selection import RandomizedSearchCV

In [57]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}

for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator = model,
                                param_distributions=params,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"Best parameters for {model_name}:")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters for KNN:
{'n_neighbors': 10}
Best parameters for RF:
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 10, 'max_depth': None}


In [58]:
models ={
    "Random Forest": RandomForestRegressor(n_estimators=200, min_samples_split= 2, max_features= 10, max_depth= None),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=10)
}

for i in range(len(models)):
    ## take the ith model
    model = list(models.values())[i]

    ## fit the model
    model.fit(X_train, y_train)

    ## predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    ## evaluate models
    model_train_mae, model_train_mse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_r2 = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])

    print('Model performance for training set')
    print(f"MAE: {model_train_mae:.4f}")
    print(f"MSE: {model_train_mse:.4f}")
    print(f"r2: {model_train_r2:.4f}")

    print('Model performance for testing set')
    print(f"MAE: {model_test_mae:.4f}")
    print(f"MSE: {model_test_mse:.4f}")
    print(f"r2: {model_test_r2:.4f}")
    
    
    print('='*35)
    print('\n')

Random Forest
Model performance for training set
MAE: 39305.2715
MSE: 15658481181.8285
r2: 0.9807
Model performance for testing set
MAE: 99279.7967
MSE: 47524177401.1810
r2: 0.9369


KNN Regressor
Model performance for training set
MAE: 103471.6175
MSE: 132103680886.5996
r2: 0.8371
Model performance for testing set
MAE: 117487.1310
MSE: 69633846174.3837
r2: 0.9075


